# Sophos Central - DattoRMM - Inventory Comparison

# DataFrame Creation from API
## Prepare for DataFrame Creation

### Import Core Modules

In [ ]:
# data import and file manipulation
import json

# custom Modules
from ts_api_connections import datto_rmm_api
from ts_api_connections import sophos_central_api
from ts_standard_naming.std_naming import NameShaping

#data conditioning
import pandas as pd
import re
import datetime as dt

### Create report_time var, git location, export location

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary folder
dictionary_dir = f'{git_folder}/dictionaries'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# DattoRMM DataFrame Creation

In [ ]:
datto = datto_rmm_api.DattoRMM()
sophos = sophos_central_api.SophosCentral()

In [ ]:
df_datto = datto.create_devices_dataframe()

### Rename values to match Sophos : 'computer' or 'server'

In [ ]:
df_datto['type'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

# Sophos DataFrame Creation

In [ ]:
df_sophos_av = create_sophos_dataframe()

# Compare devices per client

In [ ]:
rs = NameShaping('d:/git/example_infrastructure_data_dev/dictionaries')

In [ ]:
compare_cols = [
    'siteName',
    'hostname',
    'type',
    'lastSeen',
    'system'
]

## Shape DataFrames by dropping columns that are not common and rows where the site is not in sophos

### Sophos DataFrame Shaping

In [ ]:
df_sophos_av['siteName'] = df_sophos_av['siteName'].apply(rs.client_names)

In [ ]:
df_sophos_timefix = pd.to_datetime(df_sophos_av['lastSeenAt'],unit='ns')
df_sophos_av['lastSeen'] = df_sophos_timefix.values.astype('datetime64[s]')
df_sophos_av.drop('lastSeenAt',axis=1,inplace=True)

In [ ]:
df_sophos_shaped = df_sophos_av[compare_cols]


### DattoRMM DataFrame Shaping

In [ ]:
df_datto_shaped = df_datto[['siteName','type','hostname','lastSeen','system']]

In [ ]:
df_datto_timefix = pd.to_datetime(df_datto_shaped['lastSeen'], unit='ms')
#df_datto_shaped.drop('lastSeen',axis=1,inplace=True)
df_datto_shaped['lastSeen'] = df_datto_timefix

In [ ]:
df_datto_shaped = df_datto_shaped[df_datto_shaped['siteName'].isin(list(df_sophos_shaped['siteName'].unique()))]

## Compare DataFrames

In [ ]:
diff = pd.concat([df_datto_shaped,df_sophos_shaped]).drop_duplicates(subset=(['siteName','hostname']),ignore_index=True,keep=False)

In [ ]:
diff.to_csv(export_folder + 'datto_sophos_diff_' + str(current_time) + '.csv',index=False)

# Testing Area Below - Working on fixing output names to match Sophos Descriptions

In [ ]:
df_sophos_test = df_sophos_shaped

In [ ]:
def tell_me_about(s):
    uv = s.encode("utf-8")
    #print(uv)
    uv = str(uv).replace("b\'","").replace("\'","").replace("\\xe2\\x80\\x99","'").replace("\\xe2\\x80\\xb2","'")
    # print(uv)
df_sophos_test['shapedHostname'] = df_sophos_test[df_sophos_shaped['siteName'] == 'example']['hostname'].apply(tell_me_about)

In [ ]:
df_sophos_test[df_sophos_shaped['siteName'] == 'example'].to_csv('.csv',index =False)

In [ ]:
df_sophos_test = df_sophos_shaped

In [ ]:
def tell_me_about(s):
    uv = s.encode("utf-8")
    #print(uv)
    uv = str(uv).replace("b\'","").replace("\'","").replace("\\xe2\\x80\\x99","'").replace("\\xe2\\x80\\xb2","'")
    # print(uv)
df_sophos_test['shapedHostname'] = df_sophos_test[df_sophos_shaped['siteName'] == 'example']['hostname'].apply(tell_me_about)

In [ ]:
df_sophos_test[df_sophos_shaped['siteName'] == 'example'].to_csv('.csv',index =False)